# This is the code which solves the Static Traffic Assignment Problem

In [1]:
import numpy as np

In [14]:
network = 'data/I210'

In [30]:
debug = False

## 1. We load the graph and the demand
Both graph and demand are in csv file, we load them

In [31]:
graph = np.loadtxt(network + '_net.csv', delimiter=',', skiprows=1)
demand = np.loadtxt(network + '_od.csv', delimiter=',', skiprows=1)

In [32]:
# in the case where there is only one o-d, then demand is interpret as a single row and not as a matrix
try:
    demand.shape[1]
except:
    demand = np.array([demand])
nb_ods = int(demand.shape[0])
if debug:
    print(demand)

# in the case where the index of the od pairs does not begin by 0, we rename the od pairs
first_index_od = min(np.min(graph[:,1]), np.min(graph[:,2]))
graph[:,1] = graph[:,1]-first_index_od
graph[:,2] = graph[:,2]-first_index_od
demand[:,0] = demand[:,0] - first_index_od
demand[:,1] = demand[:,1] - first_index_od
if debug:
    print(first_index_od)
    print(graph)

Then, we define the function which gives the travel time as a function of the flow

In [34]:
# TO DO ADD THE CAPACITY
def travel_time(f, c = -1):
    if c == -1:
        c = [float('inf') for i in range(len(f))] # this might be to much, to do once we have everything done
    tt_tmp = graph[:,3] + graph[:,4]*f + graph[:,5]*(f**2) + graph[:,6]*(f**3) + graph[:,7]*(f**4)
    tt_tmp = [tt_tmp[i] if f[i]<c[i] else float('inf') for i in range(len(f))]
    return tt_tmp

In [35]:
nb_links = int(np.max(graph[:,0])+1)
nb_nodes = int(max(np.max(graph[:,1]), np.max(graph[:,2]))+1)
if debug:
    print("nb nodes = " + str(nb_nodes))

## 2. We compute the all or nothing flow allocation

In [36]:
import scipy

In [37]:
from scipy.sparse.csgraph import dijkstra

To use the Dijkstra's algorithm class of scipy we need to define the adjacent matrix of the graph

In [38]:
def update_travel_time(tt):
    for i in range(graph.shape[0]):
        # G[int(graph[i][1])].getcol(int(graph[i][2])).toarray().reshape(1)[0]
        # G[int(graph[i][1])][int(graph[i][2])] = tt[i]
        G[int(graph[i][1]),int(graph[i][2])] = tt[i]

In [39]:
G = np.zeros(shape=(nb_nodes,nb_nodes))
update_travel_time(travel_time(np.zeros(nb_links))) # , [1,1,-1,1,1]))
# the following line make the matrix sparse, one can use G.toarray() to do the opposite
G = scipy.sparse.csr_matrix(G)
print(G)

  (0, 1)	1.38
  (0, 6)	1.15
  (1, 2)	1.38
  (1, 7)	1.14
  (2, 3)	1.68
  (2, 8)	1.13
  (3, 4)	1.26
  (3, 9)	1.05
  (4, 5)	2.25
  (4, 10)	1.04
  (5, 11)	1.09
  (6, 0)	1.15
  (6, 7)	0.48
  (6, 12)	0.36
  (7, 1)	1.14
  (7, 8)	0.48
  (7, 13)	0.39
  (8, 2)	1.13
  (8, 9)	0.58
  (8, 14)	0.38
  (9, 3)	1.05
  (9, 10)	0.43
  (9, 15)	0.48
  (10, 4)	1.04
  (10, 11)	0.77
  (10, 16)	0.5
  (11, 5)	1.09
  (11, 17)	0.47
  (11, 19)	0.22
  (12, 6)	0.36
  (12, 13)	1.38
  (13, 7)	0.39
  (13, 14)	1.38
  (14, 8)	0.38
  (14, 15)	1.68
  (15, 9)	0.48
  (15, 16)	1.25
  (16, 10)	0.5
  (16, 17)	2.22
  (17, 11)	0.47
  (18, 6)	0.29


In [40]:
class path:
    def __init__(self,links):
        self.links = links
        self.flow = 0
    def add_flow(self, flow):
        self.flow += flow
    def __eq__(self, other):
        return np.all(self.links == other.links)
    def __hash__(self):
        return hash(np.sum([hash(self.links[i]*(nb_links**i)) for i in range(len(self.links))]))
    def __str__(self):
        return str(self.flow) + " is on " + str(self.links) 

In [41]:
paths_used = {}
print(paths_used)
p = path([0, 1, 2, 3])
paths_used[hash(p)] = (p, 0)
print(paths_used)
p = path([0, 1, 1, 3])
paths_used[hash(p)] = (p, 0)
print(paths_used)
p = path([0, 0, 2, 3])
paths_used[hash(p)] = (p, 0)
print(paths_used)

{}
{210166: (<__main__.path object at 0x111ba7710>, 0)}
{210166: (<__main__.path object at 0x111ba7710>, 0), 208485: (<__main__.path object at 0x111ba7320>, 0)}
{210166: (<__main__.path object at 0x111ba7710>, 0), 208485: (<__main__.path object at 0x111ba7320>, 0), 210125: (<__main__.path object at 0x111ba7780>, 0)}


Now let's compute the all or nothing allocation

In [64]:
# computing the all or nothing flow
paths_used = {}

def all_or_nothing():
    global k
    paths_used_tmp = {}
    faon = np.zeros(nb_links)
    # using scipy to compute dijkstra
    dist_matrix, return_predecessors = dijkstra(G, return_predecessors = True)
    """faon = np.zeros(shape = nb_links)"""
    for i in range(nb_ods):
        o_tmp = int(demand[i][0])
        d_tmp = int(demand[i][1])
        flow_tmp = demand[i][2]

        node_tmp = d_tmp
        links = []
        while node_tmp != o_tmp:
            node_tmp_d = return_predecessors[o_tmp][node_tmp]
            link_tmp = list(G.todok().keys()).index((node_tmp_d, node_tmp)) #I don't know how much time it takes, should I stock the list of keys in memory?
            links.insert(0, link_tmp)
            faon[link_tmp] += flow_tmp
            node_tmp = node_tmp_d
        
        p = path(links)
        p.add_flow(flow_tmp)
        print(p)
        if hash(p) in paths_used:
            continue
            # print(str(p) + " is already in paths_used" + str(paths_used[hash(p)]))
        else:
            print(k)
            paths_used_tmp[hash(p)] = (p, k)
            k = k+1
            
    paths_used.update(paths_used_tmp)
    return faon, paths_used_tmp

We define the line search

In [65]:
k = 0 
all_or_nothing()
print(paths_used)

25000.0 is on [13, 15, 30, 33, 19, 4, 5, 7, 9, 26, 40]
0
{545491554577279201: (<__main__.path object at 0x111ba7978>, 0)}


In [66]:
# TO DO: rewrite this function
def potential(graph, f):
    # this routine is useful for doing a line search
    # computes the potential at flow assignment f
    links = int(np.max(graph[:, 0]) + 1)
    g = np.copy(
        graph.dot(np.diag([1., 1., 1., 1., 1 / 2., 1 / 3., 1 / 4., 1 / 5.])))
    x = np.power(f.reshape((links, 1)), np.array([1, 2, 3, 4, 5]))
    return np.sum(np.einsum('ij,ij->i', x, g[:, 3:]))


def line_search(f, res=20):
    # on a grid of 2^res points bw 0 and 1, find global minimum
    # of continuous convex function
    # here we do a bisection
    d = 1. / (2**res - 1)
    l, r = 0, 2**res - 1
    while r - l > 1:
        if f(l * d) <= f(l * d + d):
            return l * d
        if f(r * d - d) >= f(r * d):
            return r * d
        # otherwise f(l) > f(l+d) and f(r-d) < f(r)
        m1, m2 = (l + r) / 2, 1 + (l + r) / 2
        if f(m1 * d) < f(m2 * d):
            r = m1
        if f(m1 * d) > f(m2 * d):
            l = m2
        if f(m1 * d) == f(m2 * d):
            return m1 * d
    return l * d

Now let run the Frank-Wolf's algorithm with a line search to find alpha

In [76]:
eps=1e-8
k = 0
paths_used = {}
f = np.zeros(nb_links)
update_travel_time(travel_time(f))

f, paths_aux = all_or_nothing()
print(f)
update_travel_time(travel_time(f))
print(paths_used)
path_matrix = np.zeros(len(paths_aux))

for val in paths_aux.values():
    print(val[1])
    path_matrix[val[1]] = val[0].flow

print(path_matrix)

for i in range(1000):
    if i % 100 == 0:
        print(i)
    faon, paths_tmp = all_or_nothing() 
    s = line_search(lambda a: potential(graph, (1. - a) * f + a * faon))
    print("s: " + str(s))
    if s < eps:
        break
    path_matrix = np.append(path_matrix, np.zeros(len(paths_used)-path_matrix.shape[0]))
    path_matrix = (1-s) * path_matrix
    for val in paths_tmp.values():
        path_matrix[val[1]] += s * val[0].flow
    f = (1. - s) * f + s * faon
    
    # also update the path flow
    ## TO DO ##
    update_travel_time(travel_time(f))
print(path_matrix)

25000.0 is on [13, 15, 18, 21, 24, 27, 40]
0
[    0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
     0.     0.     0. 25000.     0. 25000.     0.     0. 25000.     0.
     0. 25000.     0.     0. 25000.     0.     0. 25000.     0.     0.
     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
 25000.]
{193201583558: (<__main__.path object at 0x111ee7d30>, 0)}
0
[25000.]
0
25000.0 is on [13, 15, 30, 33, 35, 37, 39, 28, 40]
1
s: 0.34728394273189
25000.0 is on [13, 15, 2, 3, 5, 7, 9, 26, 40]
2
s: 0.2508300101583961
25000.0 is on [13, 15, 30, 33, 19, 4, 5, 7, 9, 26, 40]
3
s: 0.0015559606286323943
25000.0 is on [13, 15, 30, 33, 19, 4, 5, 7, 9, 26, 40]
s: 0.0
[12205.86061467  6494.24715617  6260.99321345    38.89901572]


In [81]:
for p in paths_used.values():
    index = p[1]
    p[0].add_flow(path_matrix[index] - p[0].flow)
    print(str(path_matrix[index]) + " is on " + str(p[0]))

print(demand)
print(G)
print(f)



12205.86061466584 is on 12205.86061466584 is on [13, 15, 18, 21, 24, 27, 40]
6494.247156165595 is on 6494.247156165595 is on [13, 15, 30, 33, 35, 37, 39, 28, 40]
6260.993213452754 is on 6260.993213452755 is on [13, 15, 2, 3, 5, 7, 9, 26, 40]
38.89901571580986 is on 38.8990157158114 is on [13, 15, 30, 33, 19, 4, 5, 7, 9, 26, 40]
[[1.8e+01 1.9e+01 2.5e+04]]
  (0, 1)	1.38
  (0, 6)	1.15
  (1, 2)	5089361032701062.0
  (1, 7)	4204254766144356.5
  (2, 3)	9231559.039423268
  (2, 8)	4271910701258548.0
  (3, 4)	1.26
  (3, 9)	3969474545417235.0
  (4, 5)	2.25
  (4, 10)	3931670025937070.5
  (5, 11)	1.09
  (6, 0)	1.15
  (6, 7)	0.48
  (6, 12)	3.375e+17
  (7, 1)	1.14
  (7, 8)	720000000000000.5
  (7, 13)	0.39
  (8, 2)	1.13
  (8, 9)	49434806976970.27
  (8, 14)	2088090.735107644
  (9, 3)	1.05
  (9, 10)	36649943103615.9
  (9, 15)	0.48
  (10, 4)	1.04
  (10, 11)	65628967883219.164
  (10, 16)	0.5
  (11, 5)	4120692623337891.5
  (11, 17)	2.5037025085318704e+16
  (11, 19)	1502689170669.2395
  (12, 6)	15368411972

In [61]:
list(G.todok().keys()).index((7, 13))

41


In [96]:
print("[", end="")
for i in range(len(travel_time(f))):
    print(max_float, end=", ")
print()

[1.7976931348623157e+308, 1.7976931348623157e+308, 1.7976931348623157e+308, 1.7976931348623157e+308, 1.7976931348623157e+308, 1.7976931348623157e+308, 1.7976931348623157e+308, 1.7976931348623157e+308, 1.7976931348623157e+308, 1.7976931348623157e+308, 1.7976931348623157e+308, 1.7976931348623157e+308, 1.7976931348623157e+308, 1.7976931348623157e+308, 1.7976931348623157e+308, 1.7976931348623157e+308, 1.7976931348623157e+308, 1.7976931348623157e+308, 1.7976931348623157e+308, 1.7976931348623157e+308, 1.7976931348623157e+308, 1.7976931348623157e+308, 1.7976931348623157e+308, 1.7976931348623157e+308, 1.7976931348623157e+308, 1.7976931348623157e+308, 1.7976931348623157e+308, 1.7976931348623157e+308, 1.7976931348623157e+308, 1.7976931348623157e+308, 1.7976931348623157e+308, 1.7976931348623157e+308, 1.7976931348623157e+308, 1.7976931348623157e+308, 1.7976931348623157e+308, 1.7976931348623157e+308, 1.7976931348623157e+308, 1.7976931348623157e+308, 1.7976931348623157e+308, 1.7976931348623157e+308,

In [57]:
print(graph)
print(paths_used)

[[0.0000e+00 0.0000e+00 1.0000e+00 1.3800e+00 0.0000e+00 0.0000e+00
  0.0000e+00 3.3120e+00]
 [1.0000e+00 0.0000e+00 6.0000e+00 1.1500e+00 0.0000e+00 0.0000e+00
  0.0000e+00 2.7600e+00]
 [2.0000e+00 1.0000e+00 2.0000e+00 1.3800e+00 0.0000e+00 0.0000e+00
  0.0000e+00 3.3120e+00]
 [3.0000e+00 1.0000e+00 7.0000e+00 1.1400e+00 0.0000e+00 0.0000e+00
  0.0000e+00 2.7360e+00]
 [4.0000e+00 2.0000e+00 3.0000e+00 1.6800e+00 0.0000e+00 0.0000e+00
  0.0000e+00 4.0320e+00]
 [5.0000e+00 2.0000e+00 8.0000e+00 1.1300e+00 0.0000e+00 0.0000e+00
  0.0000e+00 2.7120e+00]
 [6.0000e+00 3.0000e+00 4.0000e+00 1.2600e+00 0.0000e+00 0.0000e+00
  0.0000e+00 3.0240e+00]
 [7.0000e+00 3.0000e+00 9.0000e+00 1.0500e+00 0.0000e+00 0.0000e+00
  0.0000e+00 2.5200e+00]
 [8.0000e+00 4.0000e+00 5.0000e+00 2.2500e+00 0.0000e+00 0.0000e+00
  0.0000e+00 5.4000e+00]
 [9.0000e+00 4.0000e+00 1.0000e+01 1.0400e+00 0.0000e+00 0.0000e+00
  0.0000e+00 2.4960e+00]
 [1.0000e+01 5.0000e+00 1.1000e+01 1.0900e+00 0.0000e+00 0.0000e+00
  

### The following is old

In [95]:
max_float = sys.float_info.max
print(max_float)

1.7976931348623157e+308


In [5]:
# graph_dict gives the line of the graph matrix corresponding to the destination d and the origin o
graph_dict = {}
for i in range(graph.shape[0]):
    try: 
        graph_dict[int(graph[i][1])]
    except:
        graph_dict[int(graph[i][1])] = {}
    graph_dict[int(graph[i][1])][int(graph[i][2])] = int(graph[i][0])
    
# print(graph_dict)
# Here we remove graph_dict
del graph_dict